In [1]:
import os
import numpy as np
import glob
import PIL.Image as Image

# pip install torchsummary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
import torch.optim as optim
from time import time
from torch.utils.data import random_split

import selectivesearch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import clear_output
import xml.etree.ElementTree as ET
import json
import cv2
import pickle

In [2]:
def calculate_iou(box1, box2):
    # Convert (x_min, y_min, width, height) to (x_min, y_min, x_max, y_max)
    box1_x_min, box1_y_min, box1_x_max, box1_y_max = box1

    box2_x_min, box2_y_min, box2_width, box2_height = box2
    box2_x_max = box2_x_min + box2_width
    box2_y_max = box2_y_min + box2_height

    # Calculate intersection coordinates
    inter_x_min = max(box1_x_min, box2_x_min)
    inter_y_min = max(box1_y_min, box2_y_min)
    inter_x_max = min(box1_x_max, box2_x_max)
    inter_y_max = min(box1_y_max, box2_y_max)

    # Calculate intersection area
    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    intersection_area = inter_width * inter_height

    # Calculate union area
    box1_area = (box1_x_max - box1_x_min) * (box1_y_max - box1_y_min)
    box2_area = box2_width * box2_height
    union_area = box1_area + box2_area - intersection_area

    # Compute the IoU
    if union_area == 0:
        return 0
    iou = intersection_area / union_area
    return iou


In [4]:
data_path = "/zhome/c9/4/202487/Potholes/"

f_splits = open(os.path.join(data_path, "splits.json"))
data_splits = json.load(f_splits)
train_split = data_splits["train"]
test_split = data_splits["test"]

i = 0
image_dict = {}
for filename in train_split:

    small_filename = filename
    filename = os.path.join(data_path, "annotated-images/", filename)
    
    tree = ET.parse(filename)
    root = tree.getroot()
    boxes_list = []

    for boxes in root.iter('object'):
        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        boxes_list.append([xmin, ymin, xmax, ymax])

    jpg_filename = filename.replace("xml", "jpg")
    image = cv2.imread(jpg_filename)

    for (x, y, w, h) in boxes_list:
        cv2.rectangle(image, (x, y), (w, h), (255, 0, 0), 2)
    
    # Display the image with bounding boxes
    #plt.imshow(image)
    #plt.show()

    #new_height = int(image.shape[1] / 4)
    # Calculate a new width
    #new_width = int(image.shape[0] / 2)
    # Resize the image to the new dimensions
    #resized_image = cv2.resize(image, (new_width, new_height))
    
    # Display the resized image using Matplotlib
    #plt.imshow(resized_image)
    #plt.show()

    img_lbl, regions = selectivesearch.selective_search(
    image)

    # Initialize an empty set to store selected region proposals
    # candidates = set()
    
    output_image = image.copy()

    j = 0
    for bbox in boxes_list:
        for region in regions:
            box2 = region['rect']
            iou = calculate_iou(bbox, box2)

            if iou > 0.7:
                (x, y, w, h) = region['rect']
                

                if image_dict.get(small_filename) is None:
                    image_dict[small_filename] = [np.array([x,y,w,h,1])]
                else:
                    image_dict[small_filename].append(np.array([x,y,w,h,1]))
                #cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
            elif iou < 0.15:
                (x, y, w, h) = region['rect']
                if image_dict.get(small_filename) is None:
                    image_dict[small_filename] = [np.array([x,y,w,h,0])]
                else:
                    image_dict[small_filename].append(np.array([x,y,w,h,0]))
            
    #plt.imshow(output_image)
    #plt.show()
with open("regions.pkl", "wb") as file_save:
    pickle.dump(image_dict, file_save)


    """for region in regions:
        (x, y, w, h) = region['rect']
        cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 1)

    
    
    # Display the result  
    plt.imshow(output_image)
    plt.show()"""

        
        



/zhome/c9/4/202487/env2/lib/python3.10/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
